In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import brier_score_loss # permite ver el la calidad de la calibraion (dif cuadratica entre estimacion y prob)
from sklearn.calibration import CalibratedClassifierCV #calibra el modelo
from sklearn.calibration import calibration_curve #crea la curva de calibracion
from sklearn.metrics import brier_score_loss
from sklearn.metrics import classification_report# permite ver la precision del modelo
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
ranking=pd.read_csv("/content/data_rank.csv")

In [ ]:
ranking_away=pd.read_csv("/content/data_rank.csv")

In [ ]:
base=pd.read_csv("/content/results.csv")


In [ ]:
data=pd.pivot_table(ranking,values='Puntos',columns='Pais')

In [ ]:
data.head()

Pais,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,...,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wales,Yemen,Zambia,Zimbabwe
Puntos,1023.04,1354.11,1489.1,900.27,1035.2,1155.9,790.74,1117.01,1773.88,1233.93,...,1627.48,1638.71,1309.98,985.62,1404.37,1226.65,1569.82,1023.52,1267.41,1138.56


In [ ]:
ranking=ranking.rename(columns={'Pais':'home_team','Puntos':'Puntos home'})
ranking_away=ranking_away.rename(columns={'Pais':'away_team','Puntos':'Puntos away'})



In [ ]:
ranking.head()


,home_team,Puntos home
0,Brazil,1841.30
1,Belgium,1816.71
2,Argentina,1773.88
3,France,1759.78
4,England,1728.47


In [ ]:
ranking_away.head()

,away_team,Puntos away
0,Brazil,1841.30
1,Belgium,1816.71
2,Argentina,1773.88
3,France,1759.78
4,England,1728.47


In [ ]:
base=pd.merge(base,ranking,on='home_team',how="inner")

In [ ]:
base=pd.merge(base,ranking_away,on='away_team',how="inner")

In [ ]:
base.loc[(base.date<'2021'),'date']=None

In [ ]:
base.shape

(37681, 11)

In [ ]:
base.isnull().sum()
base=base.dropna(axis=0)
base.shape

(1785, 11)

In [ ]:
base.loc[(base.home_score>base.away_score),'winner']='home_team'
base.loc[(base.home_score<base.away_score),'winner']='away_team'
base.loc[(base.home_score==base.away_score),'winner']='Empate'


In [ ]:
base.isnull().sum()

date           0
home_team      0
away_team      0
home_score     0
away_score     0
tournament     0
city           0
country        0
neutral        0
Puntos home    0
Puntos away    0
winner         0
dtype: int64

In [ ]:
base=base.drop(['city','country','neutral','tournament'],axis=1) #'home_score','away_score','date'


In [ ]:
factor=pd.factorize(base['winner'])# la funcion factorizacion de pandas factoriza la columna winner, crea factores y definiciones de los mismos


In [ ]:
base.winner=factor[0]#asigna los factores

In [ ]:
definitions=factor[1]#guarda las definiciones

In [ ]:
print(base.winner.head())

108    0
192    0
193    1
292    2
305    1
Name: winner, dtype: int64


In [ ]:
print(definitions)

Index(['away_team', 'home_team', 'Empate'], dtype='object')


In [ ]:
base1=base

In [ ]:
base1.shape

(1785, 8)

In [ ]:
base1.head(10)

,date,home_team,away_team,home_score,away_score,Puntos home,Puntos away,winner
108,2022-11-29,Wales,England,0,3,1569.82,1728.47,0
192,2021-09-02,Hungary,England,0,4,1492.20,1728.47,0
193,2022-06-04,Hungary,England,1,0,1492.20,1728.47,1
292,2022-06-07,Germany,England,1,1,1650.21,1728.47,2
305,2022-09-23,Italy,England,1,0,1726.14,1728.47,1
386,2021-09-08,Poland,England,1,1,1548.59,1728.47,2
467,2021-03-28,Albania,England,0,2,1354.11,1728.47,0
491,2021-11-15,San Marino,England,0,10,762.22,1728.47,0
500,2021-07-03,Ukraine,England,0,4,1536.99,1728.47,0
507,2021-10-09,Andorra,England,0,5,1035.20,1728.47,0


In [ ]:
train,test=train_test_split(base1,test_size=0.1,random_state=123)

In [ ]:
scaler=StandardScaler()

In [ ]:
y_train=train.loc[:,('winner')]
x_train=train.loc[:,('Puntos home','Puntos away')]

In [ ]:
y_test=test.loc[:,('winner')]
x_test=test.loc[:,('Puntos home','Puntos away')]

In [ ]:
X=base1.loc[:,('Puntos home','Puntos away')]#

#con base.iloc[i,j] se accede mediante posiciones
#con base.loc[i,j] se accede mediante nombres

In [ ]:
Y=base1.loc[:,('winner')]#


In [ ]:
random_forest=RandomForestClassifier(n_estimators=1000,max_depth=500,random_state=10)#maxfeatures decorrealciona ,max_features=2


In [ ]:
modelo=random_forest.fit(x_train,y_train)

In [ ]:
print(random_forest.feature_importances_)

[0.51184297 0.48815703]


In [ ]:

print(classification_report(y_test,modelo.predict(x_test)))

              precision    recall  f1-score   support

           0       0.54      0.69      0.60        42
           1       0.78      0.69      0.73       100
           2       0.22      0.22      0.22        37

    accuracy                           0.59       179
   macro avg       0.51      0.53      0.52       179
weighted avg       0.61      0.59      0.60       179



In [ ]:
modelo_calibrado=CalibratedClassifierCV(modelo,cv=2,method='isotonic') # permite reducir la sobreestimacion e infraestimacion, 

In [ ]:
modelo_calibrado.fit(x_train,y_train)


CalibratedClassifierCV(base_estimator=RandomForestClassifier(max_depth=500,
                                                             n_estimators=1000,
                                                             random_state=10),
                       cv=2, method='isotonic')

In [ ]:

print(classification_report(y_test,modelo_calibrado.predict(x_test)))

              precision    recall  f1-score   support

           0       0.53      0.71      0.61        42
           1       0.74      0.88      0.80       100
           2       0.67      0.05      0.10        37

    accuracy                           0.67       179
   macro avg       0.64      0.55      0.50       179
weighted avg       0.67      0.67      0.61       179



accuracy es la precisión del modelo si bien no se relaciona directamente con la calibracion, mejoró. La calibración perrmite que no se sobreestimen las probabilidades o se subestimen

In [ ]:
x=data.loc[:,'South Korea']


In [ ]:
prueba=pd.DataFrame([[data.Argentina,data.France]],columns=['Puntos home','Puntos away'])
#prueba=np.array(prueba)
#winner_away_winner tiene un valor de 1 cuando gana visitante y  0 cuando el ganador es local
# array(probx1 de valor 0,probx2 de valor1), el x1 es la probaliidad del ganador local y x2 del ganador visitante

In [ ]:
modelo_calibrado.predict_proba(prueba)


array([[0.35526063, 0.37177977, 0.2729596 ]])

In [ ]:
reverse_factor=dict(zip(range(3),definitions))
reverse_factor
#np.vectorize()(reverse_factor), vistitante, local, empate

{0: 'away_team', 1: 'home_team', 2: 'Empate'}

Gana Paises bajos con 50% de probabilidad o empeate con 27% de probabilidad 

Gana Argentina con 51% de probabilidad 

Gana Francia 

Gana Inglaterra 

Gana japon 
 
Gana portugal 

Gana Brasil 

Gana Marruecos 

#Cuartos de final
Gana Argentina a Paises Bajos (incierto)

Gana Brasil a japón

Gana Francia contra Inglaterra

Gana marruecos contra portugal

#Semifinal
*SI queda Argentina le gana a Brasil

Gana Francia contra Marruecos

#Final
Argentina vs Francia- gana Argentina